In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [ ]:
pd.read_csv("magic04.data")

In [ ]:
cols = ["fLength","fWidth","fSize","fConc","fConc1","fAsym","fM3Long","fM3Trans","fAlpha","fDist","class"]
df = pd.read_csv("magic04.data",names=cols)
df.head()

 here from "flength" to "class" called feature, feature are just thing that we are going to pass our model in order to help us predict the label,which in this case class column.

 here sample 0 has 10 different feature,so i have 10 different values that i can pass into some model.

In [ ]:
df["class"].unique()      #g-gamma h-hydra

In [ ]:
df["class"]=(df['class']=='g').astype(int)

In [ ]:
df.head()

In [ ]:
df

In [ ]:
for labels in cols[:-1]:
  plt.hist(df[df["class"]==1][labels],color='blue',label='gamma',alpha=0.7,density=True)
  plt.hist(df[df["class"]==0][labels],color='red',label='hydron',alpha=0.7,density=True)
  plt.title(labels)
  plt.ylabel("Probability")
  plt.xlabel(labels)
  plt.legend()
  plt.show()

# Train Validate Test

In [ ]:
train, validate, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])      # splitting the dataframe, (df ...) this will shafapal my data, 0.6 means 60% data to validation, 0.8  means 80% data to test

if u seen in df.head() flength has over 100 but in fconc we have less than 100 ; the scale of numbers is way off and somethimes it will effect our results. so we need to scale this relative to mean and deviation of that specific column.

In [ ]:
def scale_dataset(dataframe):
  X = dataframe[dataframe.cols[:-1]].values
  y = dataframe[dataframe.cols[-1]].values

  # import standardScalar

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  # now we'll create the whole data as one 2d array to do that we need hstack
  # we are stacking x and y
  # numpy is very particular about dimensions
  # here x is 2d but y 1d(vector), now to reshape y to 2d, we need call reshape
  # and also need to pass dimensions
  # here (-1,1) means make this 2d array where -1 means len(y)

  data = np.hstack((X,np.reshape(y,(-1,1))))

  return data, X, y

In [ ]:
print(len(train[train["class"]==1]))  # gamma
print(len(train[train["class"]==0]))

here we have 7k gamma and 4k hydra, this will become a issue, so we need to oversample our training data sets means to increase the number of values, so they match with each other.

do reslove this we can import RandomOverSampler

In [ ]:
def scale_dataset(dataframe, oversampler=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversampler:
    ros = RandomOverSampler()
    X,y = ros.fit_resample(X,y)   # says take less class and keep sampling from their to increase the dataset of that smaller class, so that they can match

  data = np.hstack((X,np.reshape(y,(-1,1))))

  return data, X, y

In [ ]:
train, X_train, y_train = scale_dataset(train, oversampler= True)

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [ ]:
sum(y_train == 1)

In [ ]:
sum(y_train == 0)

why switching to false for validate and test bcz, i have a data that i haven't seen yet, how does my sample perform on those and i don't want to oversample those.I want ot know if i have a random set of data that's unlabeled, can i trust my model. that's why not oversampling.

In [ ]:
train, X_train, y_train = scale_dataset(train, oversampler=True)
validate, X_validate, y_vaildate = scale_dataset(validate, oversampler=False)
test, X_test, y_test = scale_dataset(test, oversampler=False)

now we have our data properly formeted now will see different models.

# KNN or K Nearest Neighbors

Now we use packages from sklearn, why we use packages, so we don't have to manually code all these things by ourself.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=1) # asks how many neighbors we want here i gave only one.
knn_model.fit(X_train,y_train)

In [ ]:
y_pred = knn_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
print(classification_report(y_pred,y_test))

accuracy is 82%, what it is saying is we just look at what each of these new points what it's closed to then we got 82% accuracy, which means how many do we get right versus how many total are there.

Now precision is we what saying class 0 and class 1, go to these site
https://en.wikipedia.org/wiki/Precision_and_recall

in the diagram, on left everything we know is actually trully +ve that we labeled +ve in our dataset and in right side is trully -ve.

Now in circle we have things +ve were labeled +ve by our model. on left side they should have been +ve but they were labelled as -ve by our model.

so precision is saying out of all the one labeled as positive, how many of them are true positive and recall saying out of all the ones labelled as +ve, how many do we actually get right.

f1-score is a combination of precision and recall score, we will be looking into this bcz we have unbalanced test data set.

#Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb_model = GaussianNB()
nb_model.fit(X_train,y_train)

In [ ]:
y_pred = nb_model.predict(X_test)

In [ ]:
print(classification_report(y_pred,y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr=LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_pred=lr.predict(X_test)

In [ ]:
print(classification_report(y_pred,y_test))

# Neural N/W

In [8]:
# example for nn

import tensorflow as tf
import numpy as np

# eg data y:2x+1

X = np.array([0,1,2,3,4],dtype=float)   # x is the input num
y = np.array([1,3,5,7,9],dtype=float)   # y is the output we want our compuer to learn

# creating simple model with 1 layer, 1 neuron

model = tf.keras.Sequential([tf.keras.layers.Dense(units=1,input_shape=[1])  # 1 neuro(smallest brain possible in ML) and it takes only 1 input number and also neuro will figure out formula connecting x and y
         ])

# compile model

model.compile(optimizer='sgd',loss='mean_squared_error')  # sgd(stochastic Gradient Descent), a method that slowly adjust the model to improve and loss='mean...', measures how wrong the model's guesses are so it knows how to improve.

# Train model

model.fit(X,y,epochs=500, verbose=0)  # epochs=500, the model will look at the data 500 times to learn, every time it looks, it makes a better guess. verbose=0 means print  all the process happens in output console but only print he result.

# predict a new value

print(model.predict(np.array([10.0]))) # a/c to equation y=2x+1, the result should be 21 or closest to our true value.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
[[20.989529]]
